# 🧬 Solution_RD: Genetic Algorithm Individual for Resource Distribution

This notebook documents the `Solution_RD` class, a domain-specific implementation of the `Individual` abstract base class tailored for solving a resource distribution problem involving participants, houses, and courses. The objective balances logistical and social criteria using a flexible and pluggable design.

## 🧱 Class Overview

- `Solution_RD`: A concrete subclass of `Individual` that represents a candidate solution for assigning participants to houses and courses.
  - Representation encodes both **house allocation** and **course enrollment**.
  - Fitness balances two components:
    - **Social fitness**: Encourages participant mixing across courses.
    - **Logistic fitness**: Penalizes long distances between participant houses (based on a distance matrix).
  - Customizable fitness trade-off via `alpha` and `beta`.

## ⚙️ Parameters

- `data_matrix`: A 2D numpy array of distances between participants.
- `account_for_previous_fitness`: If `True`, tracks fitness changes over time.
- `probability_social_mutation`: Probability of performing a social mutation.
- `alpha`, `beta`: Weights controlling the importance of social vs. logistic fitness.

## ✅ Features Demonstrated

1. **Random Initialization**
   - Genome includes both house and course assignments.
   - Ensures constraints like house capacity and number of houses are respected.

2. **Fitness Evaluation**
   - Calculated as a weighted sum of:
     - Social interaction score (maximize diversity in house-course assignments).
     - Logistic penalty (minimize travel distances).
   - Supports lazy evaluation and caching.

3. **Mutation**
   - Mutation type (social or logistic) chosen probabilistically.
   - Mutates in-place.

4. **Crossover**
   - Combines genome segments from two parents (to be implemented or overridden if necessary).

5. **Parameterization**
   - Adjust evolutionary pressure via `alpha`, `beta`, and mutation probabilities.

6. **Fitness History**
   - Tracks `previous_fitness` and `next_fitness` for advanced evolutionary strategies.

## 🧠 Design Highlights

- Extends `Individual` with problem-specific logic, maintaining modularity.
- Balances domain constraints with genetic flexibility.
- Supports experimentation with different fitness weights and mutation strategies.

This class is suitable for constrained optimization tasks in social or logistical planning domains and can be integrated seamlessly into the larger genetic algorithm framework.


In [1]:
! ls

Data_ingestion.ipynb   Solution_RD_demo.ipynb solution_class.ipynb
Individual_demo.ipynb  Test.ipynb
SelMechDemo.ipynb      solution_RD.ipynb


# Data Ingestion

In [2]:
import os
import pandas as pd
os.chdir(os.pardir) # comment if you are running this more than once
data_matrix_df = pd.read_csv('Data/distance_matrix_official.csv', index_col=0)
data_matrix_np = data_matrix_df.to_numpy()
data_matrix_np.shape

(68, 68)

# Testing Features

## Create two individuals


In [3]:
from Genetic_algorithm.fitness import ResourceFitness
from Genetic_algorithm.genome import Genome
from Genetic_algorithm.solution_rd import SolutionRD

my_fitness = ResourceFitness(data_matrix_np)
my_genome = Genome()

first_solution = SolutionRD(my_fitness, Genome)
secomnd_solution = SolutionRD(my_fitness, Genome)

## Crossover


In [4]:
child1, child2 = first_solution @ secomnd_solution
print("Fitness (child):", float(child1))
print("Fitness (child):", float(child2))

Fitness (child): 3.0969878456931474
Fitness (child): 3.220288753799392


## Mutation


In [5]:
mutant = child1.copy_Individual()
mutant.mutation()
print("original Individual:", child1.genome)
print("Mutated Individual:", mutant.genome)
print("Fitness (original):", float(child1))
print("Fitness (mutated):", float(mutant))

original Individual: 12-11002-1501247698-1385172-193-1460857431-1026-19
Mutated Individual: 12-11002-1501247698-1385172-193-1460857431-1026-19
Fitness (original): 3.0969878456931474
Fitness (mutated): 3.0969878456931474


## Iteration (generate mutated versions) (has to be resolved)

In [6]:
print("\nMultiple Mutations:")
for i, m in enumerate(child1):
    print(f"Mutation {i+1}:", m, "Fitness:", m)


Multiple Mutations:
mutating: Fitness: 3.0969878456931474
Mutation 1: None Fitness: None
mutating: Fitness: 3.0969878456931474
Mutation 2: None Fitness: None
mutating: Fitness: 3.0969878456931474
Mutation 3: None Fitness: None
mutating: Fitness: 3.0969878456931474
Mutation 4: None Fitness: None
mutating: Fitness: 3.0969878456931474
Mutation 5: None Fitness: None


## Operator Overloading

In [7]:
print("\nUsing ** Operator for Self-Crossover")
clones = child1 ** 3
for i, c1, in enumerate(clones):
    print(f"Clone {i+1}:", c1, "Fitness:", c1)
   


Using ** Operator for Self-Crossover
Clone 1: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x117506030>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x117505f10>) Fitness: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x117506030>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x117505f10>)
Clone 2: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x1174cff50>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x117518770>) Fitness: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x1174cff50>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x117518770>)
Clone 3: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x1175187d0>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x1175189e0>) Fitness: (<Genetic_algorithm.solution_rd.SolutionRD object at 0x1175187d0>, <Genetic_algorithm.solution_rd.SolutionRD object at 0x1175189e0>)


## Comparison


In [8]:
print("\nIs Individual 1 better than Individual 2?", child1 > child2)


Is Individual 1 better than Individual 2? False


## Sorting and Deduplication via Set

## Storing Unique Individuals in a Set

In [10]:
print("\n=== Deduplication with Set ===")
ind2 = child1.copy_Individual(delete_fitness=False)  # Exact copy

# Mutate a third individual to likely get a different genome
ind3 = child1.copy_Individual(delete_fitness=False)
ind3.mutation()

ind_set = {child1, ind2, ind3}

for i, ind in enumerate(ind_set):
    print(f"Unique {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")

print(f"\nTotal unique individuals: {len(ind_set)}")


=== Deduplication with Set ===
Unique 1: Genome = 12-11002-1501247698-1385172-193-1460857431-1026-19, Fitness = 3.0969878456931474

Total unique individuals: 1


# accessing the genome (removed)

In [12]:
#child1[0:4]

In [13]:
print("Before genome swap:", child1.genome.course_assignments)
child1.genome.swap_course_assignments(0,0, 1)
print("After genome swap:", child1.genome.course_assignments)

Before genome swap: [-1  5  0  1  2  4  7  6  9  8 -1  3  8  5  1  7  2 -1  9  3 -1  4  6  0
  8  5  7  4  3  1 -1  0  2  6 -1  9]
After genome swap: [-1  5  0  1  2  4  7  6  9  8 -1  3  8  5  1  7  2 -1  9  3 -1  4  6  0
  8  5  7  4  3  1 -1  0  2  6 -1  9]


In [14]:
try:    
    child1.genome.swap_course_assignments(0,0, 10000)
except IndexError as e:
    print("Error:", e)


Error: Index out of range for course assignments.


In [15]:
import os
import random
import pandas as pd
from copy import deepcopy
from typing import Callable, List
from Genetic_algorithm.selection import tournament_selection, rank_selection

from Genetic_algorithm.fitness import ResourceFitness
from Genetic_algorithm.genome import Genome
from Genetic_algorithm.solution_rd import SolutionRD

my_fitness = ResourceFitness(data_matrix_np)
my_genome = Genome()

def genetic_algorithm(
    initial_population: List[SolutionRD(my_fitness, Genome)],
    gen_count: int,
    selection_algorithm: tournament_selection,
    maximization: bool = False,
    xo_prob: float = 0.9,
    mut_prob: float = 0.2,
    elitism: bool = True,
    verbose: bool = False,
):
    """
    Evolves a population for `gen_count` generations and returns:
      • best_ind: the best individual seen *across all* generations
      • best_fitness_over_gens: list of best fitness per generation
    """
    # Initialize population
    population = initial_population
    best_fitness_over_gens: List[float] = []

    # determine best_of function
    if maximization:
        best_of = lambda pop: max(pop, key=lambda ind: ind.calculate_fitness())
    else:
        best_of = lambda pop: min(pop, key=lambda ind: ind.calculate_fitness())

    # Seed the global best from initial population
    global_best = deepcopy(best_of(population))

    for gen in range(1, gen_count + 1):
        if verbose:
            print(f"--- Generation {gen} ---")

        new_pop: List[SolutionRD(my_fitness, Genome)] = []
        # elitism: carry over last generation's best
        if elitism:
            new_pop.append(deepcopy(best_of(population)))

        # create offspring
        while len(new_pop) < len(population):
            p1 = selection_algorithm(population, maximization)
            p2 = selection_algorithm(population, maximization)
            if random.random() < xo_prob:
                off1, off2 = p1.crossover(p2)
            else:
                off1, off2 = deepcopy(p1), deepcopy(p2)
            if random.random() < mut_prob:
                off1 = off1.mutation()
            if random.random() < mut_prob:
                off2 = off2.mutation()
            new_pop.append(off1)
            if len(new_pop) < len(population):
                new_pop.append(off2)

        population = new_pop

        # track best of this generation
        this_best = best_of(population)
        this_fit  = this_best.calculate_fitness()
        best_fitness_over_gens.append(this_fit)

        # update global best if improved
        if maximization:
            if this_fit > global_best.calculate_fitness():
                global_best = deepcopy(this_best)
        else:
            if this_fit < global_best.calculate_fitness():
                global_best = deepcopy(this_best)

        if verbose:
            print(f"Best this gen: {this_fit}, Global best: {global_best.calculate_fitness()}")

    # return the individual with the best fitness across all gens, plus the trace
    return global_best, best_fitness_over_gens

In [22]:
# Example: create an initial population of 10 individuals
initial_population = [SolutionRD(my_fitness, Genome),SolutionRD(my_fitness, Genome),SolutionRD(my_fitness, Genome)]

fg, gv = genetic_algorithm(
    initial_population=initial_population,
    gen_count=20,
    selection_algorithm=tournament_selection,
    maximization=False,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=False,
)

AttributeError: 'NoneType' object has no attribute 'calculate_fitness'